<h2>Exercise 1 - Which of the following is/are TRUE about gradient boosting trees?</h2>

**Answer:** Letter F, which is both B and C

<h2>Exercise 2</h2>

**Answer:** I would choose scenario 3 since both training and validation error are low, and the depth of the tree likely wouldn't overfit the data. Scenario 4 with the higher validation error probably means that it does not react well to new data

<h2>Exercise 3 - In this course, we have covered two boosting frameworks. What is the main difference
between AdaBoost and Gradient Boosting? Be specific.</h2>

**Answer:** AdaBoost weighs previous misclassified observations by weighting them larger. Gradient Boosting fits the weak learners to the residuals/errors of the previous learners. Gradient boosting trains on minimizing the loss function

<h2>Exercise 4 - Does the gradient boosting classifier model always outperform the random forest model?
Explain.</h2>

**Answer:** No, it depends on the size of the dataset and parameter tuning. They both have their advantages like RF being less likely to overfit, while GB can learn from its own mistakes. 

<h2>Exercise 5 - Which of the following ensemble learning model does not include a learning rate
parameter?</h2>

**Answer:** A and B - RandomForestClassifier and ExtraTreesClassifier

<h2>Exercise 6 - Use framingham.csv datafile below</h2>

In [1]:
#6a - create a data-frame called heart. Remove the observations with missing values.

import pandas as pd
import numpy as np

heart = pd.read_csv('framingham(4).csv')
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [8]:
### 6b ###

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score, accuracy_score 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

#Predictors: age, totChol, sysBP, BMI, heartRate, glucose
#Target: TenYearCHD.
X = heart[['age', 'totChol' , 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

#Empty lists to store recall and accuracy results in
RF_recall = list()
RF_accuracy = list()

ET_recall = list()
ET_accuracy = list()

Ada_recall = list()
Ada_accuracy = list()

GB_recall = list()
GB_accuracy = list()

HGB_recall = list()
HGB_accuracy = list()

LGBM_recall = list()
LGBM_accuracy = list()


## Repeat 100 times ##
for i in range(0, 100):
    
    #Split data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    ### Random Forest Classifier ###
    RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    
    #Predict
    RF_pred = RF_md.predict_proba(X_test)[:, 1]
    
    #Labels with 10% cutoff
    RF_labels = np.where(RF_pred < 0.1, 0, 1)
    RF_recall.append(recall_score(Y_test, RF_labels))
    RF_accuracy.append(accuracy_score(Y_test, RF_labels))


    ### Extra Trees Classifier ###
    ET_md = ExtraTreesClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    
    #Predict
    ET_pred = ET_md.predict_proba(X_test)[:, 1]
    
    #Labels with 10% cutoff
    ET_labels = np.where(ET_pred < 0.1, 0, 1)
    ET_recall.append(recall_score(Y_test, ET_labels))
    ET_accuracy.append(accuracy_score(Y_test, ET_labels))


    ### AdaBoost ###
    Ada_md = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 3), 
                                n_estimators = 500,
                                learning_rate = 0.01).fit(X_train, Y_train)
    
    #Predict
    Ada_pred = Ada_md.predict_proba(X_test)[:, 1]
    
    #Labels with 10% cutoff
    Ada_labels = np.where(Ada_pred < 0.1, 0, 1)
    Ada_recall.append(recall_score(Y_test, Ada_labels))
    Ada_accuracy.append(accuracy_score(Y_test, Ada_labels))

    
    ### Gradient Boosting ###
    GB_md = GradientBoostingClassifier(max_depth = 3,
                                       n_estimators = 500, 
                                       learning_rate = 0.01).fit(X_train, Y_train)
    
    #Predict
    GB_pred = GB_md.predict_proba(X_test)[:, 1]

    #Labels with 10% cutoff
    GB_labels = np.where(GB_pred < 0.1, 0, 1)
    GB_recall.append(recall_score(Y_test, GB_labels))
    GB_accuracy.append(accuracy_score(Y_test, GB_labels))


    ### Hist Gradient Boosting ###
    HGB_md = HistGradientBoostingClassifier(max_depth = 3,
                                            max_iter = 500, 
                                            learning_rate = 0.01).fit(X_train, Y_train)
    
    #Predict
    HGB_pred = HGB_md.predict_proba(X_test)[:, 1]

    #Labels with 10% cutoff
    HGB_labels = np.where(HGB_pred < 0.1, 0, 1)
    HGB_recall.append(recall_score(Y_test, HGB_labels))
    HGB_accuracy.append(accuracy_score(Y_test, HGB_labels))


    ### LGBM ###
    LGBM_md = LGBMClassifier(max_depth = 3,
                             n_estimators = 500, 
                             learning_rate = 0.01).fit(X_train, Y_train)
    
    #Predict
    LGBM_pred = LGBM_md.predict_proba(X_test)[:, 1]

    #Labels with 10% cutoff
    LGBM_labels = np.where(LGBM_pred < 0.1, 0, 1)
    LGBM_recall.append(recall_score(Y_test, LGBM_labels))
    LGBM_accuracy.append(accuracy_score(Y_test, LGBM_labels))

[LightGBM] [Info] Number of positive: 445, number of negative: 2479
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000804 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 822
[LightGBM] [Info] Number of data points in the train set: 2924, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152189 -> initscore=-1.717536
[LightGBM] [Info] Start training from score -1.717536
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [12]:
#Average accuracy and recall of each model above

print('RF recall: ', np.mean(RF_recall))
print('RF accuracy: ', np.mean(RF_accuracy), '\n')

print('ET recall: ', np.mean(ET_recall))
print('ET accuracy: ', np.mean(ET_accuracy), '\n')

print('Ada recall: ', np.mean(Ada_recall))
print('Ada accuracy: ', np.mean(Ada_accuracy), '\n')

print('GB recall: ', np.mean(GB_recall))
print('GB accuracy: ', np.mean(GB_accuracy), '\n')

print('HGB recall: ', np.mean(HGB_recall))
print('HGB accuracy: ', np.mean(HGB_accuracy), '\n')

print('LGBM recall: ', np.mean(LGBM_recall))
print('LGBM accuracy: ', np.mean(LGBM_accuracy))

RF recall:  0.8515178571428571
RF accuracy:  0.45355191256830607 

ET recall:  0.9905357142857144
ET accuracy:  0.18465846994535515 

Ada recall:  0.9929464285714288
Ada accuracy:  0.15784153005464482 

GB recall:  0.8088392857142859
GB accuracy:  0.5090573770491803 

HGB recall:  0.8116964285714287
HGB accuracy:  0.49945355191256835 

LGBM recall:  0.8100892857142857
LGBM accuracy:  0.49975409836065565


From the above, I would choose HGB model to predict TenYearCHD. Although, the RF, GB & LGBM models are very similar in accuracy and recall. The ET and ADA models have very high recall but very low accuracy

### 6c ###

None of the models meet the 80% recall and accuracy requirement. You could tweak the n_estimators, learning_rate, and max_depth to try and improve the results